<a href="https://colab.research.google.com/github/salarbalou/Data_Analysis_Projects/blob/main/2DHeatConduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default='plotly_dark'
from scipy.optimize import newton_krylov

In [ ]:
nr, ntheta = 25, 27
dr, dtheta = 1./(nr-1), np.pi/2/(ntheta-1)
r, theta = np.mgrid[1:4:(nr*1j), 0:np.pi/2:(ntheta*1j)]

Tsol = np.zeros((nr,ntheta))

Tsol[0,:], Tsol[-1,:] = 130, 150
Tsol[:,0], Tsol[:,-1] = 135, 145


def residual(T):
    Tsol[1:-1,1:-1]=T
    d2r = np.zeros_like(T)
    d2theta = np.zeros_like(T)
    
    d2r = ((r[1:-1,1:-1]+dr/2) *(Tsol[2:,1:-1]-Tsol[1:-1,1:-1]) - (r[1:-1,1:-1]-dr/2)*(Tsol[1:-1,1:-1]-Tsol[:-2,1:-1]))/dr**2 
    d2theta = (Tsol[1:-1,2:] - 2*Tsol[1:-1,1:-1] + Tsol[1:-1,:-2])/dtheta**2/r[1:-1,1:-1]


    return d2r + d2theta

# solve
guess = np.zeros_like(Tsol[1:-1,1:-1])
Tsol[1:-1,1:-1] = newton_krylov(residual, guess, method='lgmres', verbose=0)


In [ ]:

x,y = r*np.cos(theta), r*np.sin(theta)

fig = make_subplots()
for i in np.arange(nr):
     fig.add_trace(go.Scatter3d(x=x[i,:], y=y[i,:], z=Tsol[i,:], mode='lines', line_color='gray'))

for j in np.arange(ntheta):
     fig.add_trace(go.Scatter3d(x=x[:,j], y=y[:,j], z=Tsol[:,j], mode='lines', line_color='gray'))

fig.add_trace(go.Surface(x=x,y=y,z=Tsol))

fig.update_layout(width=800, height=800, scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='T'),
                  showlegend=False)